In [27]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [28]:
%pip install catboost

In [29]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from sklearn.pipeline import make_pipeline


voting_reg = VotingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('xgboost_01', xgb.XGBRegressor(max_depth=6, n_estimators=60, random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(max_depth=8, n_estimators=40, random_state=42)),
        ('catboost_01',Catboost(iterations=10, depth=10, learning_rate=1, random_state=42)),
        ('catboost_02',Catboost(iterations=20, depth=8, learning_rate=1, random_state=42)),
    ]
)
voting_reg.fit(X_train, y_train)

0:	learn: 1.0674234	total: 377ms	remaining: 3.39s
1:	learn: 1.0607193	total: 731ms	remaining: 2.92s
2:	learn: 1.0598742	total: 1.07s	remaining: 2.51s
3:	learn: 1.0593128	total: 1.44s	remaining: 2.17s
4:	learn: 1.0589240	total: 1.8s	remaining: 1.8s
5:	learn: 1.0582659	total: 2.16s	remaining: 1.44s
6:	learn: 1.0578656	total: 2.54s	remaining: 1.09s
7:	learn: 1.0572039	total: 2.92s	remaining: 730ms
8:	learn: 1.0568427	total: 3.29s	remaining: 366ms
9:	learn: 1.0564089	total: 3.68s	remaining: 0us
0:	learn: 1.0682203	total: 182ms	remaining: 3.46s
1:	learn: 1.0623745	total: 366ms	remaining: 3.3s
2:	learn: 1.0603842	total: 552ms	remaining: 3.13s
3:	learn: 1.0601403	total: 730ms	remaining: 2.92s
4:	learn: 1.0597593	total: 907ms	remaining: 2.72s
5:	learn: 1.0592751	total: 1.06s	remaining: 2.48s
6:	learn: 1.0590332	total: 1.25s	remaining: 2.32s
7:	learn: 1.0587974	total: 1.41s	remaining: 2.12s
8:	learn: 1.0586414	total: 1.6s	remaining: 1.96s
9:	learn: 1.0584606	total: 1.79s	remaining: 1.79s
10:	le

VotingRegressor(estimators=[('lin_reg', LinearRegression()),
                            ('xgboost_01',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          inte...
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=40,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('catboost_01',
                             <catboost.core.CatBoostRegressor object at 0x7f2dfd7949d0>),
                            ('catboost_02',
                             <catboost.core.CatBoostRegressor object at 0x7f2dfd794fd0>)])

In [30]:
from sklearn.metrics import root_mean_squared_error
y_pred = voting_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0517175409318047

In [31]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [32]:
y_pred = voting_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0606745535650375

In [33]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']
y_pred = voting_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('voting_reg_06.csv', index=False)